In [93]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime
#prevents warnings from being printed
import warnings
warnings.filterwarnings('ignore')

In [2]:
requests = pd.read_csv('requests.csv')
requests[:5]

,User,Title,Status,URL,Jurisdiction,Jurisdiction ID,Jurisdiction Level,Jurisdiction State,Agency,Agency ID,...,Tracking Number,Embargo,Days since submitted,Days since updated,Projects,Tags,Price,Date Submitted,Date Due,Date Done
0,PNWPals,02.10.18 IAP and OPAAR,No Responsive Documents,https://www.muckrock.com/foi/seattle-69/021018...,Seattle,69,Local,Washington,Seattle Police Department,227,...,P024303-022618,False,1303.0,1240,NaN,NaN,0.0,2018-02-23 05:00:00+00:00,2018-03-02,2018-04-27 00:00:00+00:00
1,Woods,0215 Memphis - Kristen Smith,No Responsive Documents,https://www.muckrock.com/foi/memphis-319/0215-...,Memphis,319,Local,Tennessee,University Of Memphis - Memphis,15225,...,NaN,False,214.0,169,NaN,NaN,0.0,2021-02-16 16:41:44.333874+00:00,2021-02-25,2021-04-02 17:57:40.391587+00:00
2,null_name,02/29/16 - SLCPD Abdi Mohamed Protest Action P...,No Responsive Documents,https://www.muckrock.com/foi/salt-lake-city-35...,Salt Lake City,359,Local,Utah,Salt Lake City Police Department,4223,...,NaN,False,2010.0,1986,A Protest Project,NaN,0.0,2016-03-18 04:00:00+00:00,2016-04-01,2016-04-11 00:00:00+00:00
3,MichelleMalkin,#04-5812 public records request,Completed,https://www.muckrock.com/foi/midwest-city-2705...,Midwest City,27056,Local,Oklahoma,Midwest City Police Department,12077,...,NaN,False,957.0,927,NaN,NaN,0.0,2019-02-04 18:58:22.499172+00:00,NaN,2019-03-06 16:04:14.465141+00:00
4,EmmaBest,100-18762 Harry Hay,Awaiting Response,https://www.muckrock.com/foi/united-states-of-...,United States of America,10,Federal,United States of America,Federal Bureau of Investigation,10,...,NaN,False,1115.0,934,Freedom of LGBTQIA+ Information,NaN,0.0,2018-08-30 12:05:11.174398+00:00,2018-09-28,NaN


In [3]:
requests.describe()

,Jurisdiction ID,Agency ID,Days since submitted,Days since updated,Price
count,72502.000000,72502.000000,72501.000000,72502.000000,7.250200e+04
mean,2115.750434,4371.423188,1426.428877,1232.194726,2.749678e+04
std,5739.671447,5923.852666,906.070096,924.606720,4.245461e+06
min,1.000000,1.000000,0.000000,0.000000,0.000000e+00
25%,10.000000,133.000000,701.000000,432.000000,0.000000e+00
50%,80.000000,1407.000000,1284.000000,1089.000000,0.000000e+00
75%,336.000000,6204.000000,2079.000000,1858.000000,0.000000e+00
max,33667.000000,28772.000000,4139.000000,4124.000000,6.600000e+08


In [4]:
police_reqs = requests[requests['Agency'].str.contains("(?i)police|sheriff|public safety", case = False)]

In [5]:
non_police_reqs = requests[~requests['Agency'].str.contains("(?i)police|sheriff|public safety", case = False)]

In [6]:
len(police_reqs) + len(non_police_reqs) == len(requests)

True

In [35]:
police_reqs.to_csv('police_reqs.csv', index = False)
non_police_reqs.to_csv('non_police_reqs.csv', index = False)

In [7]:
by_agency = police_reqs[['Agency']].value_counts().to_frame().rename(columns = {0: "Requests Received"}).reset_index()
#by_agency.to_csv('numRequests_byAgency.csv', index = False)

In [8]:
#top 5 agencies with most requests
#check csv for full list
by_agency.head()

,Agency,Requests Received
0,New York City Police Department,556
1,Chicago Police Department,508
2,Boston Police Department,369
3,Seattle Police Department,326
4,Massachusetts State Police,241


In [9]:
by_jurlevel = police_reqs["Jurisdiction Level"].value_counts().to_frame().reset_index().rename(columns = {'index': 'Jurisdiction Level', 'Jurisdiction Level': 'Requests Received'})
#by_jurlevel.to_csv('numRequests_byJurisdiction.csv', index = False)

In [10]:
#num requests by jurisdiciton
by_jurlevel

,Jurisdiction Level,Requests Received
0,Local,19063
1,State,2279
2,Federal,43


### How quickly on average do law enforcement agencies respond to requests versus non-law enforcement agencies?

In [11]:
#convert to datetime object
police_reqs['Date Submitted'] = pd.to_datetime(police_reqs['Date Submitted'])
police_reqs['Date Due'] = pd.to_datetime(police_reqs['Date Due'])
police_reqs['Date Done'] = pd.to_datetime(police_reqs['Date Done'])

In [12]:
non_police_reqs['Date Submitted'] = pd.to_datetime(non_police_reqs['Date Submitted'])
non_police_reqs['Date Due'] = pd.to_datetime(non_police_reqs['Date Due'])
non_police_reqs['Date Done'] = pd.to_datetime(non_police_reqs['Date Done'])

In [13]:
#calculate means, exclude non values
police_reqs['Days Till Completion'] = police_reqs['Date Done'] - police_reqs['Date Submitted']
police_time = np.mean(police_reqs['Days Till Completion'][~np.isnan(police_reqs['Days Till Completion'])])

In [14]:
non_police_reqs['Days Till Completion'] = non_police_reqs['Date Done'] - non_police_reqs['Date Submitted']
non_police_time = np.mean(non_police_reqs['Days Till Completion'][~np.isnan(non_police_reqs['Days Till Completion'])])

In [33]:
#calculate proportion of nan values
prop_nonresponse_police = len(police_reqs[np.isnan(police_reqs['Days Till Completion'])])/len(police_reqs)
prop_nonresponse_nonpolice = len(non_police_reqs[np.isnan(non_police_reqs['Days Till Completion'])])/len(non_police_reqs)

In [39]:
print("Law enforcement response time to requests: ", police_time.round('1min') ,"\nNon-Law Enforcement response time: ", non_police_time.round('1min'))
print("\nMeans calculated with nonresponse entries omitted. \nProportion of nonresponse entries for police records: ", 
      round(prop_nonresponse_police, 2), "\nProportion of nonresponse non-police records: ", round(prop_nonresponse_nonpolice, 2))

Law enforcement response time to requests:  96 days 13:57:00 
Non-Law Enforcement response time:  151 days 11:34:00

Means calculated with nonresponse entries omitted. 
Proportion of nonresponse entries for police records:  0.33 
Proportion of nonresponse non-police records:  0.33


### How quickly on average does each individual law enforcement agency respond to requests? 

Issue with the dataset is that there are some entries that never had the "Date Done" column filled out or there were errors with the entry where the date was entered incorrectly. Errors include:
- Completing the entry on the same day but instead of entering the "Date Done" to be the next day, the same day was entered leading to a negtive time difference. 
- Date is just entered incorrectly. 

For a simple fix, the ```replaceNegatives``` function is implemented to change the "Days Till Completion" value to be positive.

In [134]:
#removed records where submission hasn't been completed 
completed_police_reqs = police_reqs[~pd.isnull(police_reqs['Days Till Completion'])]

In [138]:
completed_police_reqs[completed_police_reqs['Days Till Completion'] <= timedelta(days = -1)]

,User,Title,Status,URL,Jurisdiction,Jurisdiction ID,Jurisdiction Level,Jurisdiction State,Agency,Agency ID,...,Embargo,Days since submitted,Days since updated,Projects,Tags,Price,Date Submitted,Date Due,Date Done,Days Till Completion
21631,dmehrotra,Dutchess County etc. Facial Recognition Docume...,No Responsive Documents,https://www.muckrock.com/foi/poughkeepsie-2241...,Poughkeepsie,22419,Local,New York,Poughkeepsie Police Department,4357,...,False,758.0,751,NaN,NaN,0.0,2019-08-22 15:43:22.204299+00:00,2019-08-29,2019-08-13 04:56:00+00:00,-10 days +13:12:37.795701
22254,graih,El Paso IRIS,Partially Completed,https://www.muckrock.com/foi/el-paso-339/el-pa...,El Paso,339,Local,Texas,El Paso County Sheriff's Office,8949,...,False,1524.0,1391,NaN,NaN,0.0,2017-07-17 04:00:00+00:00,2017-07-31,2017-07-08 00:00:00+00:00,-10 days +20:00:00
37059,EmmaBest,J20 Protest Footage (United States Capitol Pol...,Rejected,https://www.muckrock.com/foi/united-states-of-...,United States of America,10,Federal,United States of America,United States Capitol Police,4243,...,False,1676.0,1676,NaN,NaN,0.0,2017-02-15 05:00:00+00:00,2017-03-16,2017-02-01 00:00:00+00:00,-15 days +19:00:00
56238,lucaswaldron,ProPublica: Incident Report Request (Haines Ci...,Rejected,https://www.muckrock.com/foi/haines-10345/prop...,Haines,10345,Local,Florida,Haines City Police Department,11094,...,False,1294.0,1282,NaN,NaN,0.0,2018-03-04 05:00:00+00:00,NaT,2018-03-02 00:00:00+00:00,-3 days +19:00:00


In [139]:
#function takes in a timedelta object and adds to it to take care of data entry errors
def replaceNegatives(x):
    if x <= timedelta(days = 0):
        if x.days != -1:
            x += timedelta(days = 2*abs(x.days))
        else:
            x += timedelta(1)
        return x
    return x

In [140]:
completed_police_reqs['Days Till Completion'] = completed_police_reqs['Days Till Completion'].apply(replaceNegatives)

In [133]:
avg_time = completed_police_reqs.groupby('Agency').agg({'Days Till Completion': pd.Series.mean}).reset_index().rename(columns = {'Days Till Completion': 'Avg Days Till Completion'})
avg_time.to_csv('Avg Time of Response by Agency.csv', index = False)

#first 5 records, check csv for full list
avg_time.head()

,Agency,Avg Days Till Completion
0,Abbevile Police Department,31 days 20:00:00
1,Abbeville Police Department,97 days 20:37:43.811949
2,Aberdeen Police Department,30 days 19:03:00.422427250
3,Abilene Texas Police Department,252 days 20:49:24.204696500
4,Abington Police Department,19 days 07:08:21.447689111


In [25]:
police_reqs.head()

,User,Title,Status,URL,Jurisdiction,Jurisdiction ID,Jurisdiction Level,Jurisdiction State,Agency,Agency ID,...,Embargo,Days since submitted,Days since updated,Projects,Tags,Price,Date Submitted,Date Due,Date Done,Days Till Completion
0,PNWPals,02.10.18 IAP and OPAAR,No Responsive Documents,https://www.muckrock.com/foi/seattle-69/021018...,Seattle,69,Local,Washington,Seattle Police Department,227,...,False,1303.0,1240,NaN,NaN,0.0,2018-02-23 05:00:00+00:00,2018-03-02,2018-04-27 00:00:00+00:00,62 days 19:00:00
2,null_name,02/29/16 - SLCPD Abdi Mohamed Protest Action P...,No Responsive Documents,https://www.muckrock.com/foi/salt-lake-city-35...,Salt Lake City,359,Local,Utah,Salt Lake City Police Department,4223,...,False,2010.0,1986,A Protest Project,NaN,0.0,2016-03-18 04:00:00+00:00,2016-04-01,2016-04-11 00:00:00+00:00,23 days 20:00:00
3,MichelleMalkin,#04-5812 public records request,Completed,https://www.muckrock.com/foi/midwest-city-2705...,Midwest City,27056,Local,Oklahoma,Midwest City Police Department,12077,...,False,957.0,927,NaN,NaN,0.0,2019-02-04 18:58:22.499172+00:00,NaT,2019-03-06 16:04:14.465141+00:00,29 days 21:05:51.965969
12,JPatBrown,10/22/15 Somerville Police Report,Withdrawn,https://www.muckrock.com/foi/somerville-8/1022...,Somerville,8,Local,Massachusetts,Somerville Police Department,42,...,False,2149.0,2027,NaN,NaN,0.0,2015-10-31 04:00:00+00:00,NaT,NaT,NaT
13,invisibleinstitute,1026812 documents,Completed,https://www.muckrock.com/foi/chicago-169/10268...,Chicago,169,Local,Illinois,Chicago Police Department,503,...,False,1665.0,1643,NaN,NaN,0.0,2017-02-26 05:00:00+00:00,2017-03-03,2017-03-20 00:00:00+00:00,21 days 19:00:00


In [37]:
grouped = police_reqs.groupby(['Agency', 'Status']).count().reset_index()[['Agency', 'Status', 'User']].rename(columns = {'User': 'Count'})
nyc = grouped[grouped['Agency'] == "New York City Police Department"]
nyc

,Agency,Status,Count
4487,New York City Police Department,Awaiting Acknowledgement,2
4488,New York City Police Department,Awaiting Appeal,13
4489,New York City Police Department,Awaiting Response,47
4490,New York City Police Department,Completed,100
4491,New York City Police Department,Fix Required,2
4492,New York City Police Department,No Responsive Documents,88
4493,New York City Police Department,Partially Completed,4
4494,New York City Police Department,Payment Required,2
4495,New York City Police Department,Rejected,249
4496,New York City Police Department,Withdrawn,49
